In [0]:
%python
## Import libraries
import pyspark as ps
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os
import boto3
import datetime
import pandas as pd

In [0]:
#convert sucess sql qyery to dataframe
Takhzyro_success_query = "select * from com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed where Sent_To_Veeva = current_date and Not_Sent_To_Veeva_Reason is NULL and usecase_id = '511';"
Takhzyro_success_query_df = sqlContext.sql(Takhzyro_success_query).collect()
Takhzyro_success_list = []
for curr_row in Takhzyro_success_query_df: 
  Takhzyro_success_list.append (curr_row.asDict(True))
Takhzyro_success_df = pd.DataFrame(Takhzyro_success_list)

#convert Error sql qyery to dataframe
Takhzyro_error_query = "WITH t(usecase_id, hist_retention) AS (SELECT USECASE_ID,hist_retention from com_us_alyt_ngebox.metadata_usecase) select * from com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed where DATE(created_date) >=  date_add(current_date, -(select hist_retention from t where t.usecase_id = usecase_id )) and Sent_To_Veeva is null and processed = 1 and Not_Sent_To_Veeva_Reason is not null and usecase_id = '511'"
Takhzyro_error_query_df = sqlContext.sql(Takhzyro_error_query).collect()
Takhzyro_error_list = []
for curr_row in Takhzyro_error_query_df: 
  Takhzyro_error_list.append (curr_row.asDict(True))
Takhzyro_error_df = pd.DataFrame(Takhzyro_error_list)

In [0]:
# Set the S3 bucket and folder path
s3_bucket = 'tpc-aws-ted-dev-edpp-alyt-com-us-east-1'
Takhzyro_s3_folder_path_arc = 'NGE_Box/IN_N_OUT/OUT/RD/IENGAGE_PUBLISH/Takhzyro_Leads_Labs/ARCHIVE/'
Takhzyro_s3_folder_path_STG = 'NGE_Box/IN_N_OUT/OUT/RD/IENGAGE_PUBLISH/Takhzyro_Leads_Labs/STG/'

# Get the current date and time
folder_name = datetime.datetime.now().strftime("%Y-%m-%d")

# Create the S3 client
s3_client = boto3.client('s3')

# Create the CSV file name using the current date
now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
Takhzyro_success_csv_file_name = f'Veeva_Suggestion_File_Takhzyro_Leads_labs_{now}.csv'
Takhzyro_error_csv_file_name = f'Error_Veeva_Suggestion_File_Takhzyro_Leads_labs_{now}.csv'

# Convert the DataFrame to CSV data
Takhzyro_success_csv_data = Takhzyro_success_df.to_csv(index=False)
Takhzyro_error_csv_data = Takhzyro_error_df.to_csv(index=False)

# Write the dynamic CSV file to the S3 bucket folder for success and error files
s3_client.put_object(Bucket=s3_bucket, Key=Takhzyro_s3_folder_path_arc + folder_name + '/' + Takhzyro_success_csv_file_name, Body=Takhzyro_success_csv_data)
s3_client.put_object(Bucket=s3_bucket, Key=Takhzyro_s3_folder_path_arc + folder_name + '/' + Takhzyro_error_csv_file_name, Body=Takhzyro_error_csv_data)

# Remove the existing CSV files in the STG path
response = s3_client.list_objects_v2(Bucket=s3_bucket, Prefix=Takhzyro_s3_folder_path_STG)
for obj in response.get('Contents', []):
    if obj['Key'].endswith('.csv'):
        s3_client.delete_object(Bucket=s3_bucket, Key=obj['Key'])
        print(f"Deleted object: {obj['Key']}")

# Copy the new CSV file to the STG path
s3_client.put_object(Bucket=s3_bucket, Key=Takhzyro_s3_folder_path_STG + Takhzyro_success_csv_file_name, Body=Takhzyro_success_csv_data)

Deleted object: NGE_Box/IN_N_OUT/OUT/RD/IENGAGE_PUBLISH/Takhzyro_Leads_Labs/STG/Veeva_Suggestion_File_Takhzyro_Leads_labs_20230512_113725.csv
Out[3]: {'ResponseMetadata': {'RequestId': 'T3CJ8YD6T944ZQW8',
  'HostId': 'jd9/UX+kxb0+IcOvEetizNPj+h0XL6TV3V4X3RgR2pl5MnW+o/Er1v+QXPA8OUpPHDwHVj/r+Is=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'jd9/UX+kxb0+IcOvEetizNPj+h0XL6TV3V4X3RgR2pl5MnW+o/Er1v+QXPA8OUpPHDwHVj/r+Is=',
   'x-amz-request-id': 'T3CJ8YD6T944ZQW8',
   'date': 'Sat, 13 May 2023 16:48:27 GMT',
   'x-amz-version-id': 'z.Q._wB0RhZ0Cab20F5iCNC57pMOIpM.',
   'x-amz-server-side-encryption': 'aws:kms',
   'x-amz-server-side-encryption-aws-kms-key-id': 'arn:aws:kms:us-east-1:263789222982:key/253920df-b84a-47ca-a26e-ace8b712a886',
   'etag': '"9b994d3f6cd9b21d00dc4722b4d680b4"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"9b994d3f6cd9b21d00dc4722b4d680b4"',
 'ServerSideEncryption': 'aws:kms',
 'VersionId': 'z.Q._wB0RhZ0Cab20F5iCNC

In [0]:
class GetNgeMetadata:
   def construct_microservice_response(self):
    resp = {
      "nge_response": {
        "status": 200,
        "body": "s3 file load completed"
      }
    }
    self.response = resp

In [0]:
metadata_obj = GetNgeMetadata()
metadata_obj.construct_microservice_response()
dbutils.notebook.exit(
  metadata_obj.response
)

{'nge_response': {'status': 200, 'body': 's3 file load completed'}}